In [1]:
%matplotlib inline


# Topic extraction with Non-negative Matrix Factorization and Latent Dirichlet Allocation


This is an example of applying :class:`sklearn.decomposition.NMF` and
:class:`sklearn.decomposition.LatentDirichletAllocation` on a corpus
of documents and extract additive models of the topic structure of the
corpus.  The output is a list of topics, each represented as a list of
terms (weights are not shown).

Non-negative Matrix Factorization is applied with two different objective
functions: the Frobenius norm, and the generalized Kullback-Leibler divergence.
The latter is equivalent to Probabilistic Latent Semantic Indexing.

The default parameters (n_samples / n_features / n_components) should make
the example runnable in a couple of tens of seconds. You can try to
increase the dimensions of the problem, but be aware that the time
complexity is polynomial in NMF. In LDA, the time complexity is
proportional to (n_samples * iterations).




In [43]:
from os import listdir
from os.path import isfile, join
from os import environ
import os
from nltk.tokenize import TreebankWordTokenizer, word_tokenize
import re
import indexes
from time import time

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import langid

In [44]:
#sent_text = nltk.sent_tokenize(text) # this gives us a list of sentences
with open( os.path.join(os.environ["AAN_DIR"],"papers_text/{0}".format("W04-0203.txt"))) as f:
    good_text  = f.read().replace('-\n','')
with open( os.path.join(os.environ["AAN_DIR"],"papers_text/{0}".format("W04-0207.txt"))) as f:
    good_text2  = f.read().replace('-\n','')
with open( os.path.join(os.environ["AAN_DIR"],"papers_text/{0}".format("W04-0205.txt"))) as f:
    bad_text  = f.read().replace('-\n','')
with open( os.path.join(os.environ["AAN_DIR"],"papers_text/{0}".format("J79-1011.txt"))) as f:
    bad_text2 = f.read().replace('-\n','')   
    

In [45]:
import langid
langid.classify(bad_text)

('rw', -9756.831540584564)

In [46]:
langid.classify(good_text2)

('en', -64000.54495596886)

In [52]:
from langid.langid import LanguageIdentifier, model
identifier = LanguageIdentifier.from_modelstring(model, norm_probs=True)
identifier.classify(bad_text2)

('an', 1.0)

In [60]:
identifier.classify(bad_text2)

('an', 1.0)

In [54]:
tf_feature_names = tf_vectorizer.get_feature_names()
print(tf_feature_names)

['cb', 'ed', 'ic', 'pa']


In [74]:
tf_vectorizer = CountVectorizer(#max_df=0.95, 
                                 min_df=2,
                                input="content",
                                #max_features=n_features,
                                stop_words='english',
                                token_pattern=r"(?u)\b[a-zA-Z][a-zA-Z]+\b",
                                preprocessor = dehyphenate
                                #tokenizer=TreebankWordTokenizer().tokenize
                                )
t0 = time()
#heuristic 1: number of words in common with good text > 20
#heuristic 2 : number of words with 1 letter
tf = tf_vectorizer.fit_transform([good_text,good_text2])
tf_feature_names = tf_vectorizer.get_feature_names()
print(tf_feature_names)

['able', 'abstract', 'according', 'account', 'added', 'additional', 'agreement', 'al', 'alternative', 'analysis', 'annotated', 'annotating', 'annotation', 'annotator', 'appear', 'application', 'applications', 'applied', 'approach', 'assigned', 'assumed', 'automated', 'automatic', 'based', 'basic', 'basis', 'best', 'better', 'boundaries', 'canonical', 'case', 'category', 'class', 'classification', 'classifier', 'classifiers', 'cognition', 'combination', 'combinations', 'combined', 'compare', 'compared', 'comparing', 'comparison', 'complex', 'computational', 'conclusions', 'containing', 'content', 'context', 'contrast', 'contribution', 'corpus', 'correspond', 'corresponding', 'data', 'described', 'detailed', 'development', 'did', 'difference', 'different', 'dis', 'discourse', 'discussed', 'discussion', 'distribution', 'does', 'domain', 'earlier', 'effect', 'element', 'end', 'english', 'et', 'evidence', 'example', 'examples', 'expect', 'expected', 'experiments', 'explicit', 'extended', 'e

In [ ]:
tf_vectorizer = CountVectorizer(#max_df=0.95, 
                                input="content",
                                #max_features=n_features,
                                stop_words='english',
                                token_pattern=r"(?u)\b[a-zA-Z][a-zA-Z]+\b",
                                preprocessor = dehyphenate
                                #tokenizer=TreebankWordTokenizer().tokenize
                                )
tf = tf_vectorizer.fit_transform([bad_text2])
tf_feature_names = tf_vectorizer.get_feature_names()
print(tf_feature_names)

In [ ]:
with open( os.path.join(os.environ["AAN_DIR"],"papers_text/{0}".format("W04-0207.txt"))) as f:
    good_text2  = f.read().replace('-\n','')
good_text2

In [15]:
train_dirpath = os.path.join(os.environ["AAN_DIR"],"papers_text")
    
#train_files = [join(train_dirpath, f) for f in listdir(train_dirpath) if isfile(join(train_dirpath, f))]
#train_files = [join(train_dirpath, f) for f in listdir(train_dirpath) if isfile(join(train_dirpath, f))]
train_files = [join(environ["AAN_DIR"],"papers_text/{0}".format(fn)) for fn in 
               listdir(join(environ["AAN_DIR"],"papers_text/")) if isfile(join(train_dirpath, fn)) and "txt" in fn]
# data = []
# for file in train_files:
#     with open(file) as f:
#         doc = f.read()
#         data.append(doc.replace('-\n',''))

In [66]:
bad_pdfs = []
bad_eng = []
for train_file in train_files[:100]:
    with open(train_file) as f:
        txt = f.read()
        l, c = identifier.classify(txt)

        bad_eng.append((train_file,l,c))

In [55]:
results = []
for f in bad_pdfs:
    with open(f) as bf:
        txt = bf.read()
        results.append(identifier.classify(txt))

In [73]:
import _pickle as pkl
a = pkl.load(open("bad_pdfs.pkl","rb"))
a

[('C92-4191', 'fr', 1.0),
 ('E03-3002', 'en', 0.16946150595865334),
 ('C04-1052', 'ht', 0.7870253041711965),
 ('W06-2913', 'ht', 1.0),
 ('W12-1202', 'fr', 1.0),
 ('C82-1041', 'de', 1.0),
 ('P98-1073', 'fr', 1.0),
 ('P98-1062', 'es', 1.0),
 ('W12-4406', 'se', 0.9999999870834935),
 ('E03-1054', 'en', 0.16946150595865334),
 ('C02-1106', 'pl', 0.9999938805231214),
 ('W12-1306', 'fr', 1.0),
 ('W02-1907', 'rw', 0.999999999999549),
 ('C04-1178', 'sl', 0.8622732832732573),
 ('P00-1049', 'az', 0.999779042257573),
 ('W06-3704', 'en', 0.16946150595865334),
 ('C02-1060', 'rw', 0.9999999841756986),
 ('W04-0704', 'en', 0.16946150595865334),
 ('E03-1014', 'en', 0.16946150595865334),
 ('W12-1310', 'fr', 1.0),
 ('C69-6501', 'de', 1.0),
 ('P01-1062', 'se', 0.9999940733481725),
 ('E03-1079', 'en', 0.16946150595865334),
 ('W12-1206', 'fr', 1.0),
 ('W02-2014', 'rw', 0.9999999999999953),
 ('E03-1087', 'en', 0.16946150595865334),
 ('C02-1142', 'rw', 0.9999962892035603),
 ('W01-0726', 'rw', 0.9990578994870087

In [59]:
really_bad = []
for f in bad_pdfs:
    with open(f) as bf:
        txt = bf.read()
        ln = identifier.classify(txt)
        if(ln == fr):
            french.append(f)
        else:
            really_bad.append(ln)

'/home/ramona/Desktop/Honours-LDA/aan/papers_text/C04-1052.txt'

In [62]:
french = list(filter(lambda x: x[0] == "fr", bad_pdfs)
french

In [18]:
def dehyphenate(s):
    return s.replace('-\n','').lower()

In [26]:

ids_path = os.path.join(os.environ["AAN_DIR"],
    "release/2014/acl-metadata.txt")

fields = ["id", "authors", "title", "venue", "year","genders"]
with open(ids_path,"r", encoding="utf-8") as f:
    paper_data = f.read().split("\n\n")
    for idx,paper in enumerate(paper_data):
        values = paper.split("\n")[:len(fields)-1]

        values = dict(zip(fields,[re.search(r'{(.*?)}',s).group(1) for s in values]+[[]]))

        print(values["id"])
        break

E03-1001


In [27]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [178]:
# Use tf (raw term count) features for LDA.
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(#max_df=0.95, 
                                input='filename',
                                min_df=2,
                                #max_features=n_features,
                                stop_words='english',
                                token_pattern=r"(?u)\b[a-zA-Z][a-zA-Z]+\b",
                                preprocessor = dehyphenate
                                #tokenizer=TreebankWordTokenizer().tokenize
                                )
t0 = time()
tf = tf_vectorizer.fit_transform(train_files)
print("done in %0.3fs." % (time() - t0))
print()

Extracting tf features for LDA...
done in 253.499s.



In [176]:
tf_vectorizer.tokenizer

In [171]:
n_features = 1000
n_components = 10
n_top_words = 20

In [172]:
print("Fitting LDA models with tf features, "
      "and n_features=%d..."
      % (n_features))
lda = LatentDirichletAllocation(n_components=n_components, max_iter=20,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
t0 = time()
lda.fit(tf)
print("done in %0.3fs." % (time() - t0))


Fitting LDA models with tf features, and n_features=1000...
done in 0.332s.


In [173]:
print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)


Topics in LDA model:
Topic #0: semantic information representation pp features parses linguistic node np linguistics processing corpus analysis query task using distance predicate possible paper
Topic #1: scope disambiguation proceedings pages approach local based mention semantic mentions paper context data words parses using example joint language linguistic
Topic #2: semantic token predicate representation words np information using word data phrase used number base set results role features pp argument
Topic #3: features entity speech model figure semantic pages et al input integration using proper proceedings coreference type set scope word feature
Topic #4: linguistic information meaning processing sense domain semantic category world way features point context illustrates using model language words discourse tag
Topic #5: semantic scope mention features word words entity using al disambiguation mentions et information approach model set pages language proceedings data
Topic #6:

In [1]:
# Author: Olivier Grisel <olivier.grisel@ensta.org>
#         Lars Buitinck
#         Chyi-Kwei Yau <chyikwei.yau@gmail.com>
# License: BSD 3 clause

from __future__ import print_function
from time import time

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.datasets import fetch_20newsgroups

n_samples = 2000
n_features = 1000
n_components = 10
n_top_words = 20


def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()


# Load the 20 newsgroups dataset and vectorize it. We use a few heuristics
# to filter out useless terms early on: the posts are stripped of headers,
# footers and quoted replies, and common English words, words occurring in
# only one document or in at least 95% of the documents are removed.

print("Loading dataset...")
t0 = time()
dataset = fetch_20newsgroups(shuffle=True, random_state=1,
                             remove=('headers', 'footers', 'quotes'))
data_samples = dataset.data[:n_samples]
print("done in %0.3fs." % (time() - t0))

# Use tf-idf features for NMF.
print("Extracting tf-idf features for NMF...")
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2,
                                   max_features=n_features,
                                   stop_words='english')
t0 = time()
tfidf = tfidf_vectorizer.fit_transform(data_samples)
print("done in %0.3fs." % (time() - t0))

# Use tf (raw term count) features for LDA.
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=n_features,
                                stop_words='english')
t0 = time()
tf = tf_vectorizer.fit_transform(data_samples)
print("done in %0.3fs." % (time() - t0))
print()

# Fit the NMF model
print("Fitting the NMF model (Frobenius norm) with tf-idf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
t0 = time()
nmf = NMF(n_components=n_components, random_state=1,
          alpha=.1, l1_ratio=.5).fit(tfidf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in NMF model (Frobenius norm):")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)

# Fit the NMF model
print("Fitting the NMF model (generalized Kullback-Leibler divergence) with "
      "tf-idf features, n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
t0 = time()
nmf = NMF(n_components=n_components, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(tfidf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in NMF model (generalized Kullback-Leibler divergence):")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)

print("Fitting LDA models with tf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
lda = LatentDirichletAllocation(n_components=n_components, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
t0 = time()
lda.fit(tf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

Loading dataset...
done in 17.985s.
Extracting tf-idf features for NMF...
done in 0.570s.
Extracting tf features for LDA...
done in 0.608s.

Fitting the NMF model (Frobenius norm) with tf-idf features, n_samples=2000 and n_features=1000...
done in 1.228s.

Topics in NMF model (Frobenius norm):
Topic #0: just people don think like know time good make way really say right ve want did ll new use years
Topic #1: windows use dos using window program os drivers application help software pc running ms screen files version card code work
Topic #2: god jesus bible faith christian christ christians does heaven sin believe lord life church mary atheism belief human love religion
Topic #3: thanks know does mail advance hi info interested email anybody looking card help like appreciated information send list video need
Topic #4: car cars tires miles 00 new engine insurance price condition oil power speed good 000 brake year models used bought
Topic #5: edu soon com send university internet mit ftp 

KeyboardInterrupt: 